# Analysis of transactions.csv

Our second dataset was the transactions.csv dataset, which describes transactions from January 2020 to September 2022 within the bar. The variables it includes are: product code, amount of units sold, sale amounts, discount amount, profit, discount percentage, and percentage markup. We only kept product code, amount (rubles), and sale amount (rubles), as the remaining variables were not independent of each other. 
After cleaning the data, we plotted a heatmap of the remaining variables and found that there was some correlation with sale amount and amount.
